In [1]:
import fitz  # PyMuPDF library
import json  # JSON module


def extract_toc(file_path): # PDF ning sarlavha qismini ajratib oluvchi metod.
    pdf = fitz.open(file_path)
    toc = pdf.get_toc()

    toc_dict = {}
    previous_heading = None

    for entry in toc:
        level, title, page = entry

        if level == 1:
            if previous_heading:
                # Avvalgi sarlavha va hozirgi sarlavha orasidagi matnni tekshiramiz.
                text_between = extract_text_between_pages(
                    pdf, toc_dict[previous_heading]["page"], page
                )
                if not text_between.strip():  # Agar matn bo'sh bo'lsa
                    # Avvalgi sarlavha nomini yangilaymiz
                    toc_dict[previous_heading + ". " + title] = toc_dict.pop(previous_heading)
                    previous_heading = previous_heading + ". " + title
                else:
                    toc_dict[title] = {"page": page, "subtitles": [], "text": ""}
                    previous_heading = title
            else:
                toc_dict[title] = {"page": page, "subtitles": [], "text": ""}
                previous_heading = title
        elif level == 2:
            if toc_dict:
                last_heading = list(toc_dict.keys())[-1]
                toc_dict[last_heading]["subtitles"].append(
                    {"title": title, "page": page, "text": ""}
                )

    return toc_dict


def extract_text_between_pages(pdf, start_page, end_page):
    """
    Extract text between two pages in the PDF.
    """
    text = ""
    for page_num in range(start_page, end_page):
        page = pdf[page_num]
        text += page.get_text("text")
    return text.strip()


def extract_text_for_headings(file_path):
    pdf = fitz.open(file_path)
    toc_dict = extract_toc(file_path)

    for heading, details in toc_dict.items():
        start_page = details["page"]
        next_heading = None

        # Yakuniy sahifani aniqlash uchun keyingi sarlavha sahifasini topish.
        next_headings = list(toc_dict.keys())
        current_heading_index = next_headings.index(heading)
        if current_heading_index + 1 < len(next_headings):
            next_heading = next_headings[current_heading_index + 1]
            end_page = toc_dict[next_heading]["page"]
        else:
            end_page = pdf.page_count  # Agar bu oxirgi sarlavha bo'lsa, PDF-ning oxirigacha o'tish.

        # Sarlavhalar orasidagi matnni ajratib olish.
        toc_dict[heading]["text"] = extract_text_between_pages(pdf, start_page - 1, end_page - 1)

        # Subtitlelar uchun matnni chiqarib olish.
        for i, subtitle in enumerate(details["subtitles"]):
            sub_start_page = subtitle["page"]
            sub_end_page = details["subtitles"][i + 1]["page"] if i + 1 < len(details["subtitles"]) else end_page
            subtitle["text"] = extract_text_between_pages(pdf, sub_start_page - 1, sub_end_page - 1)

    return toc_dict


def save_to_json(data, output_file):
    with open(output_file, "w", encoding="utf-8") as json_file:
        json.dump(data, json_file, ensure_ascii=False, indent=4)


# Pdf fayl yo'li
file_path = "base.pdf"  # o'zingizning faylingiz nomiga o'zgartirsangiz bo'ladi.

# Sarlavhalar va sahifalardan matnni ajratib olish
heading_texts = extract_text_for_headings(file_path)

# Lug'atni JSON formatida saqlash
output_file = "structure.json"
save_to_json(heading_texts, output_file)

# Natijani lug'at shaklida chop etish
for heading, details in heading_texts.items():
    print(f"Sarlavha: {heading}, Sahifa: {details['page']}")
    print(f"    Matn: {details['text'][:100]}...")  # Faqat birinchi 100 ta belgini ko'rsatadi

    for subtitle in details["subtitles"]:
        print(f" Kichik sarlavha: {subtitle['title']}, Sahifa: {subtitle['page']}")
        print(f"  Matn: {subtitle['text'][:100]}...")  # Faqat birinchi 100 ta belgini ko'rsatadi


Sarlavha: Как пользоваться книгой, Sahifa: 11
    Matn: КАК ПОЛЬЗОВАТЬСЯ 
КНИГОЙ 
Конфигурация «Бухгалтерия для Узбекистана редакция 
3.0» обладает широкой ...
Sarlavha: Глава 1. Характеристика программы, Sahifa: 14
    Matn: ГЛАВА 1 
ХАРАКТЕРИСТИКА 
ПРОГРАММЫ 
Программный 
продукт 
включает 
технологическую 
платформу 
«1С:...
 Kichik sarlavha: Новые возможности платформы «1С:Предприятие 8.3», Sahifa: 15
  Matn: продаж и т. д. Данное прикладное решение также можно 
использовать только для ведения бухгалтерского...
 Kichik sarlavha: Учет документов, Sahifa: 16
  Matn: ...
 Kichik sarlavha: Ведение учета деятельности нескольких организаций, Sahifa: 16
  Matn: ...
 Kichik sarlavha: Поддержка разных систем налогообложения, Sahifa: 16
  Matn: Учет документов 
Основным способом отражения хозяйственных операций 
в 
учете 
является 
ввод 
докум...
 Kichik sarlavha: Поддержка учета деятельности организаций, имеющих обособленные подразделения, Sahifa: 17
  Matn: ...
 Kichik sarlavha: Учет матер